In [1]:
import pandas as pd
import requests
import os
import secrets
import sys
sys.path.append('./../cbp')
import utils

In [2]:
# READ 2006 DATA FILE 
cbp06 = pd.read_csv('./cbp06co.txt')

In [3]:
# Filter for Texas
tx06 = cbp06[cbp06.fipstate == 48]
tx06.head()

,fipstate,fipscty,naics,empflag,emp,qp1,ap,est,n1_4,n5_9,...,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,censtate,cencty
1791801,48,1,------,NaN,11591,79324,344842,935,510,205,...,7,3,2,0,0,0,0,0,74,1
1791802,48,1,11----,A,0,0,0,2,0,2,...,0,0,0,0,0,0,0,0,74,1
1791803,48,1,113///,A,0,0,0,2,0,2,...,0,0,0,0,0,0,0,0,74,1
1791804,48,1,1131//,A,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,74,1
1791805,48,1,11311/,A,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,74,1


In [4]:
# Clean up NAICS codes
tx06.naics = tx06.naics.str.replace('-', '')
tx06.naics = tx06.naics.str.replace('/', '')
tx06.naics = tx06.apply(lambda row: '00' if row.naics == '' else row.naics, axis = 1)

/home/paul/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
# Clean up County FIPS codes
def county_fips(row):
    
    fips_str = str(row['fipscty'])
    if len(fips_str) == 1:
        fips = '00' + fips_str
    elif len(fips_str) == 2:
        fips = '0' + fips_str
    else:
        fips = fips_str
        
    return fips

tx06['county'] = tx06.apply(county_fips, axis=1)

/home/paul/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
tx06.head()

,fipstate,fipscty,naics,empflag,emp,qp1,ap,est,n1_4,n5_9,...,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,censtate,cencty,county
1791801,48,1,00,NaN,11591,79324,344842,935,510,205,...,3,2,0,0,0,0,0,74,1,001
1791802,48,1,11,A,0,0,0,2,0,2,...,0,0,0,0,0,0,0,74,1,001
1791803,48,1,113,A,0,0,0,2,0,2,...,0,0,0,0,0,0,0,74,1,001
1791804,48,1,1131,A,0,0,0,1,0,1,...,0,0,0,0,0,0,0,74,1,001
1791805,48,1,11311,A,0,0,0,1,0,1,...,0,0,0,0,0,0,0,74,1,001


In [7]:
# Reduce to relevant columns

tx06 = tx06.loc[:, ['emp', 'est', 'naics', 'fipstate', 'county']]

In [8]:
tx06.head()

,emp,est,naics,fipstate,county
1791801,11591,935,00,48,001
1791802,0,2,11,48,001
1791803,0,2,113,48,001
1791804,0,1,1131,48,001
1791805,0,1,11311,48,001


In [9]:
naics2007 = utils.update_naics(years='2002-2007', old_series=tx06.naics)
naics2012 = utils.update_naics(years='2007-2012', old_series=naics2007)

In [10]:
tx06['naics2012'] = naics2012

In [11]:
tx06.head()

,emp,est,naics,fipstate,county,naics2012
1791801,11591,935,00,48,001,00
1791802,0,2,11,48,001,11
1791803,0,2,113,48,001,113
1791804,0,1,1131,48,001,1131
1791805,0,1,11311,48,001,11311


In [18]:
tx06.fipstate = tx06.fipstate.astype('str')

In [24]:
tx06.dtypes

EMP           int64
ESTAB         int64
old_naics    object
state        object
county       object
NAICS2012    object
dtype: object

In [20]:
tx06.columns = ['EMP', 'ESTAB', 'old_naics', 'state', 'county', 'NAICS2012']

In [30]:
tx06.NAICS2012.replace(to_replace='31', value='31-33', inplace=True)
tx06.NAICS2012.replace(to_replace='44', value='44-45', inplace=True)
tx06.NAICS2012.replace(to_replace='48', value='48-49', inplace=True)

In [31]:
tx06.to_csv('./texas_cbp_2006.csv', index=False)